# Introduction to Big Data Analytics - List 5
### _Wojciech Korczyński, 229949_


In [60]:
# importing libraries
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import pyspark.sql.functions as f

In [5]:
# Creating the Spark Session
spark = SparkSession.builder.appName("ML_bank_marketing").getOrCreate()

## Data loading

We load data from `.csv` file.

In [6]:
bankdata = spark.read.csv(path='bank.csv',
                         sep=',',
                         encoding='UTF-8',
                         comment=None,
                         header=True,
                         inferSchema=True)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|59 |admin.    |married|secondary|no     |2343   |yes    |no  |unknown|5  |may  |1042    |1       |-1   |0       |unknown |yes    |
|56 |admin.    |married|secondary|no     |45     |no     |no  |unknown|5  |may  |1467    |1       |-1   |0       |unknown |yes    |
|41 |technician|married|secondary|no     |1270   |yes    |no  |unknown|5  |may  |1389    |1       |-1   |0       |unknown |yes    |
|55 |services  |married|secondary|no     |2476   |yes    |no  |unknown|5  |may  |579     |1       |-1   |0       |unknown |yes    |
|54 |admin.    |married|tertiary |no     |184    |no     |no  |unknown|5  |m

In [75]:
bankdata.show(n=5, truncate=False)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|59 |admin.    |married|secondary|no     |2343   |yes    |no  |unknown|5  |may  |1042    |1       |-1   |0       |unknown |yes    |
|56 |admin.    |married|secondary|no     |45     |no     |no  |unknown|5  |may  |1467    |1       |-1   |0       |unknown |yes    |
|41 |technician|married|secondary|no     |1270   |yes    |no  |unknown|5  |may  |1389    |1       |-1   |0       |unknown |yes    |
|55 |services  |married|secondary|no     |2476   |yes    |no  |unknown|5  |may  |579     |1       |-1   |0       |unknown |yes    |
|54 |admin.    |married|tertiary |no     |184    |no     |no  |unknown|5  |m

In [74]:
pd.DataFrame(bankdata.take(5), columns=bankdata.columns)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [8]:
bankdata.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



Category class is weel-balanced:

In [10]:
bankdata.groupBy("deposit").count().show()

+-------+-----+
|deposit|count|
+-------+-----+
|     no| 5873|
|    yes| 5289|
+-------+-----+



In [11]:
bankdata.describe().show()

+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+-------+
|summary|               age|    job| marital|education|default|           balance|housing| loan| contact|               day|month|          duration|          campaign|             pdays|          previous|poutcome|deposit|
+-------+------------------+-------+--------+---------+-------+------------------+-------+-----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+-------+
|  count|             11162|  11162|   11162|    11162|  11162|             11162|  11162|11162|   11162|             11162|11162|             11162|             11162|             11162|             11162|   11162|  11162|
|   mean|41.231947679627304|   null|    null|     null|   null|1528.5385235620856|   null| null|    null

Lack of NULL values.

In [15]:
data_agg = bankdata.agg(*[f.count(f.when(f.isnull(col), col)).alias(col) for col in bankdata.columns])
data_agg.show()

+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|  0|  0|      0|        0|      0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|      0|
+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+



## Data preprocessing

It is neede to transform string data into numerical one using `StringIndexer` cause most of ML algorithms expect nummercial values. In order to avoid some hierarchy, indexed data have to be transforemd using `OneHotEncoderEstimator` which creates sparse vector. All features will be written in one vecotr column using `VectorAssembler`.

In [52]:
category_columns = ['job', 'marital', 'education', 'default','housing', 'loan', 'contact', 'month', 'poutcome']
indexers = [
    StringIndexer(inputCol=col, outputCol="{0}_indexed".format(col))
    for col in category_columns
]

In [53]:
label_string_indexer = StringIndexer(inputCol = 'deposit', outputCol = 'label')

In [54]:
encoders = OneHotEncoderEstimator(dropLast=False)\
    .setInputCols([indexer.getOutputCol() for indexer in indexers])\
    .setOutputCols(["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers])    

In [55]:
encoders.getOutputCols()

['job_indexed_encoded',
 'marital_indexed_encoded',
 'education_indexed_encoded',
 'default_indexed_encoded',
 'housing_indexed_encoded',
 'loan_indexed_encoded',
 'contact_indexed_encoded',
 'month_indexed_encoded',
 'poutcome_indexed_encoded']

In [56]:
numericCols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
assembler_inputs = [out_col for out_col in encoders.getOutputCols()] + numericCols
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

We create a `Pipeline` which performs all action needed to preprocess data.

In [64]:
stages = [indexer for indexer in indexers]
stages += [label_string_indexer]
stages += [encoders]
stages += [assembler]
pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(bankdata)
bankdata_updated = pipelineModel.transform(bankdata)

In [68]:
original_cols = bankdata.columns
selected_cols = ['label', 'features'] + original_cols
df = bankdata_updated.select(selected_cols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [73]:
df.show(n=5, truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|label|features                                                                                                                |age|job       |marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+-----+------------------------------------------------------------------------------------------------------------------------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|1.0  |(51,[3,12,15,19,22,23,26,28,40,44,45,46,47,48,49],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,59.0,2343.0,5.0,1042.0,1.0,-1.0])|59 |admin.    |married|secondary|no     |2343   |yes    |no  |unknown|5  |may  |1042    |1     